In [1]:
import monai
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract

In [2]:
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import glob
import os


In [3]:
print_config()

MONAI version: 1.4.dev2438
Numpy version: 1.26.4
Pytorch version: 2.3.1+cxx11.abi
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: d85038e6acf92febeacfc53463a7e70e35d82670
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.2.0
scikit-image version: 0.23.1
scipy version: 1.13.0
Pillow version: 9.3.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 5.2.0
TorchVision version: 0.18.1+cxx11.abi
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.0.0
pandas version: 2.2.0
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional depen

Run the below command in bash to temporary add the directory into the system 

"export MONAI_DATA_DIRECTORY=/home/max/Documents/Monai/MONAI_DATA_DIRECTORY"

In [4]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")

if directory is not None:
    os.makedirs(directory, exist_ok=True)

else:print(None)

root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

None
/tmp/tmpl1525pjw


In [5]:
data_dir = ("/home/max/Documents/Monai/MONAI_DATA_DIRECTORY/Task09_Spleen/")

In [6]:
train_images = sorted(glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_labels = sorted(glob.glob(os.path.join(data_dir, "labelsTr", "*.nii.gz")))


In [7]:
data_dicts = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)]

train_files, val_files = data_dicts[:-9], data_dicts[-9:]

In [8]:
set_determinism(seed=44)

In [9]:
train_transforms = Compose(
    [
    
    LoadImaged(keys=["image", "label"]),
    
    EnsureChannelFirstd(keys=["image", "label"]),
    
    ScaleIntensityRanged(keys=["image"],a_min=-57,a_max=164,
                         b_min=0.0,b_max=1.0,clip=True,),
    
    CropForegroundd(keys=["image", "label"], source_key="image"),
    
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    
    Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), 
             mode=("bilinear", "nearest")),
    
    RandCropByPosNegLabeld(keys=["image", "label"],label_key="label",
                           spatial_size=(96, 96, 96),pos=1,neg=1,
                           num_samples=4,image_key="image",image_threshold=0,
        
)])

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
